In [3]:
import spacy
from spacy.tokens import Doc
from spacy.matcher import Matcher
from spacy import displacy
from zipfile import ZipFile
from pathlib import Path
from seqeval.metrics import classification_report
from seqeval import scheme
from tqdm import autonotebook as tqdm
import os
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from collections.abc import Iterable


In [4]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
# REGULAR DATA IMPORT
data_dir = Path("./data/teaching-dataset")
with (data_dir / "span_extraction_text_train.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("input.txt") as f:
        sentences = [
            sentence.split("\n") for sentence in f.read().decode("utf-8").split("\n\n")
        ]
with (data_dir / "span_extraction_references_train.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("references.txt") as f:
        labels = [
            sentence.split("\n") for sentence in f.read().decode("utf-8").split("\n\n")
        ]

In [5]:
# THIS IS MY METHOD OF DATA IMPORTING (USING VS-CODE CAUSES ISSUES OTHERWISE)
parent = Path(os.getcwd()).parent
data_dir = Path(os.path.join(parent,"data/span-detection"))

with (data_dir / "span_extraction_text_train.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("input.txt") as f:
        sentences = [
            sentence.split("\n") for sentence in f.read().decode("utf-8").split("\n\n")
        ]
with (data_dir / "span_extraction_references_train.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("references.txt") as f:
        labels = [
            sentence.split("\n") for sentence in f.read().decode("utf-8").split("\n\n")
        ]

In [6]:
train_sentences, test_sentences, train_labels, test_labels =  train_test_split(sentences, labels, test_size=0.2, random_state=42)

In [44]:
# same here
class is_causal_predictor:
    def __init__(self, sentences, labels, n=None):
        self.sentences = sentences
        self.labels = labels
        self.n = n
        self.test_sentences = test_sentences
        self.test_labels = test_labels
        
        self.init_words()
        if n != None:
            self.causal_cues = self.get_causal_cues(self.n)


    def init_words(self):
        self.words = []
        self.nonCausalWords = []

        for label, sentence in zip(self.labels, self.sentences):
            if type(sentence) == list:
                sentence = ' '.join(sentence)
            if "B-EVENT" in label: #if sentence is causal
                doc = nlp(sentence)
                wordsHelp = [token.text for token in doc if not token.is_stop and not token.is_punct and token.pos_ != "NOUN" and token.pos_ != "ADJ"] 
                self.words.extend(wordsHelp) #append all words to a list if they are NOT nouns & NOT punctuation & NOT adjectives

            else: #if sentence is NOT causal
                doc = nlp(sentence)
                wordsHelp = [token.text for token in doc if not token.is_stop and not token.is_punct and token.pos_ != "NOUN" and token.pos_ != "ADJ"] 
                self.nonCausalWords.extend(wordsHelp)  
    
    # returns n best causal cues
    def get_causal_cues(self, n):
        def flatten(lis): #pretty ugly solution but we have to flatten the list since every new sentence adds "[]" which Counter can't deal with
            for item in lis:
                if isinstance(item, Iterable) and not isinstance(item, str):
                    for x in flatten(item):
                        yield x
                else:        
                    yield item
        
        def get_n_lemmata(causal_freq, n):
            # sort words
            sorted_words = np.array(causal_freq.most_common(len(causal_freq)))[:,0]
            converted_return = []
            for word in sorted_words:
                # lemmatize
                doc = nlp(str(word))
                word = " ".join([token.lemma_ for token in doc])
                if not word in converted_return:
                    converted_return.append(word)
                # break if n lemmata found
                if len(converted_return)==n:
                    break
            return converted_return

        causal_freq = Counter(self.words)
        nonCausal_freq = Counter(self.nonCausalWords)
        for word in causal_freq:
            causal_freq[word] = causal_freq[word]/(nonCausal_freq[word]+1)
        return get_n_lemmata(causal_freq, n)


    def predict_causality(self, sentence):
        if type(sentence) == list: # convert to str
            sentence = ' '.join(sentence)
        matcher = Matcher(nlp.vocab)
        pattern = [[{"LEMMA": cue}] for cue in self.causal_cues]
        matcher.add("CAUSAL", pattern)
        doc = nlp(sentence)
        matches = matcher(doc)
        return bool(matches)


    # Find best value for n given a testset and initialize causal cues according to best n
    def init_causal_cues_best_n(self, test_sentences, test_labels, step_size=20):
        new_labels = []
        for label in test_labels:
            if "B-EVENT" in label:
                new_labels.append(1)
            else:
                new_labels.append(0)

        old_f1 = 0
        f1 = 0
        n = 0
        while f1 >= old_f1:
            n = n+step_size
            predictions = []

            # predict
            self.causal_cues = self.get_causal_cues(n)
            for sentence, l in zip(test_sentences, test_labels):
                p = self.predict_causality(sentence)
                predictions.append(p)

            # evaluate
            old_f1 = f1
            tp = sum([int(p) == 1 and int(l) == 1 for p, l in zip(predictions, new_labels)])
            fp = sum([int(p) == 1 and int(l) == 0 for p, l in zip(predictions, new_labels)])
            fn = sum([int(p) == 0 and int(l) == 1 for p, l in zip(predictions, new_labels)])
            precision = tp / (tp + fp)
            recall = tp / (tp + fn)
            f1 = 2 * precision * recall / (precision + recall)

        self.f1 = old_f1
        self.n = n-20
        self.causal_cues = self.get_causal_cues(self.n)

In [45]:
is_causal_predictor = is_causal_predictor(train_sentences,train_labels, 120)

In [55]:
"raise" in is_causal_predictor.causal_cues

False

In [122]:
def predict_3(sentence):

    def compound_handler(token):
        for child in token.children:
            object_handler(child)
            if child.dep_ in  ["amod","compound","poss","det","nmod","case"]:
                predictions[child.i] = "B-EVENT"
                compound_handler(child)
            if child.dep_ in  ["acl"] and child.tag_ in ["VBG"]:
                predictions[child.i] = "B-EVENT"
                compound_handler(child)
            if child.dep_ in  ["prep"]:
                if child.text not in ["as"]:
                    predictions[child.i] = "B-EVENT"
                for prep_child in child.children:
                    object_handler(prep_child)

    def conj_handler(token):
        if token.dep_ in ["conj", "cconj", "cc"]:
            predictions[token.i] = "B-EVENT"
            compound_handler(token)
        if doc[token.i - 1].text == "-":
            predictions[token.i - 1] = "B-EVENT"
            predictions[token.i - 2] = "B-EVENT"
        # find whole conjunction
        for child in token.children:
            conj_handler(child)

    def object_handler(token):
        if token.dep_ in ["dobj", "pobj", "nsubj", "nsubjpass","advcl"] and token.pos_ not in ["PRON"]:
            predictions[token.i] = "B-EVENT"
            compound_handler(token)

            for child in token.children:
                object_handler(child)
                conj_handler(child)

    def cleanup():
        def find_closing_bracket():
            for j in range(i,len(sentence)):
                if doc[j].text == ")":
                    return j

        for token in doc:
            if token.dep_ in ["det"] and predictions[token.i - 1] != "B-EVENT":
                predictions[token.i] = "O" 

        for i, pred in enumerate(predictions):
            if doc[i].text == "(":
                closing_bracket = find_closing_bracket()
                predictions[i:closing_bracket] = "O" * (closing_bracket-i)
            if pred == "B-EVENT" or pred == "I-EVENT":
                try:
                    if predictions[i+1] == "B-EVENT":
                        predictions[i+1] = "I-EVENT"
                except:
                    pass

            

    def loop_for_verbs(allow_non_causal_cues=True):
        for token in doc:
            if token.lemma_ in causal_cues or allow_non_causal_cues:
                if token.dep_ in ["ROOT", "advcl", "relcl", "ccomp", "conj"]:
                    for child in token.children:
                        # root-child is object?
                        object_handler(child)
                        # root-child is prep
                        if child.dep_ in ["prep", "agent"]:
                            for prep_child in child.children:
                                # prep-child is object?
                                object_handler(prep_child)

                if token.dep_ == "acl":
                    for child in token.children:
                        object_handler(child)
                        if child.dep_ == "agent":
                            for agent_child in child.children:
                                object_handler(agent_child)
                        
                    for parent in token.ancestors:
                        object_handler(parent)

    doc = Doc(nlp.vocab, words=sentence)
    doc = nlp(doc)
    predictions = ["O"] * len(sentence)
    causal_cues = is_causal_predictor.causal_cues
    loop_for_verbs()
    if "B-EVENT" not in predictions:
        loop_for_verbs(allow_non_causal_cues=False)

    cleanup()
    return predictions

idx = 14
sentence = sentences[idx]
doc = Doc(nlp.vocab, words=sentence)
doc = nlp(doc)
iterator = zip(doc, predict_3(sentences[idx]), labels[idx])
for token, pred_3, label in iterator:
    print(f"{token.text :<16} {pred_3 :<8} {label :<8} {token.pos_ :<8} {token.dep_:<8} {token.tag_:<8}")

displacy.render(doc, style="dep")

AGEP             B-EVENT  B-EVENT  PROPN    nsubj    NNP     
is               O        O        AUX      ROOT     VBZ     
an               O        O        DET      det      DT      
acute            O        O        ADJ      amod     JJ      
drug             O        O        NOUN     compound NN      
eruption         O        O        NOUN     attr     NN      
characterized    O        O        VERB     acl      VBN     
by               O        O        ADP      agent    IN      
numerous         B-EVENT  O        ADJ      amod     JJ      
small            I-EVENT  O        ADJ      amod     JJ      
,                O        O        PUNCT    punct    ,       
primarily        O        O        ADV      advmod   RB      
non              B-EVENT  O        ADJ      amod     JJ      
-                I-EVENT  O        ADJ      punct    JJ      
follicular       I-EVENT  O        ADJ      amod     JJ      
,                O        O        PUNCT    punct    ,       
sterile 

In [59]:
spacy.explain("vbg")

In [123]:
predictions_1 = [predict_3(sentence) for sentence in tqdm.tqdm(test_sentences)]
print(classification_report(test_labels, predictions_1, scheme=scheme.IOB2, mode="default"))

  0%|          | 0/94 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       EVENT       0.33      0.42      0.37       294

   micro avg       0.33      0.42      0.37       294
   macro avg       0.33      0.42      0.37       294
weighted avg       0.33      0.42      0.37       294



In [ ]:
nlp = spacy.load("en_core_web_sm")
for label in nlp.get_pipe("parser").labels:
    print(label, " -- ", spacy.explain(label))

ROOT  --  root
acl  --  clausal modifier of noun (adjectival clause)
acomp  --  adjectival complement
advcl  --  adverbial clause modifier
advmod  --  adverbial modifier
agent  --  agent
amod  --  adjectival modifier
appos  --  appositional modifier
attr  --  attribute
aux  --  auxiliary
auxpass  --  auxiliary (passive)
case  --  case marking
cc  --  coordinating conjunction
ccomp  --  clausal complement
compound  --  compound
conj  --  conjunct
csubj  --  clausal subject
csubjpass  --  clausal subject (passive)
dative  --  dative
dep  --  unclassified dependent
det  --  determiner
dobj  --  direct object
expl  --  expletive
intj  --  interjection
mark  --  marker
meta  --  meta modifier
neg  --  negation modifier
nmod  --  modifier of nominal
npadvmod  --  noun phrase as adverbial modifier
nsubj  --  nominal subject
nsubjpass  --  nominal subject (passive)
nummod  --  numeric modifier
oprd  --  object predicate
parataxis  --  parataxis
pcomp  --  complement of preposition
pobj  --  ob

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/spacy/glossary.py:19: UserWarning: [W118] Term 'predet' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(Warnings.W118.format(term=term))


In [124]:
test_predictions = []
for sentence in tqdm.tqdm(test_sentences):
    test_predictions.append(predict_3(sentence))
Path("predictions_rule_based_2.txt").write_text("\n".join(map(str, test_predictions)))

  0%|          | 0/94 [00:00<?, ?it/s]

20003